In [ ]:
import os
import ast
import pandas as pd

c:\Users\AIAS\Documents\cti-model-training\venv-eda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
reports_references = pd.read_csv("data/opencti_reports_external_references.csv")

In [3]:
references_dict = {
    x["ID"]: ast.literal_eval(x["Ext. Reference URLs"])
    for _, x in reports_references.iterrows()
}

In [ ]:
from docling.document_converter import DocumentConverter

converter = DocumentConverter()

def extract_report(source):
    result = converter.convert(source)
    return result.document.export_to_markdown()

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

# 1. Initialize Selenium WebDriver (adjust path to your webdriver)
driver = webdriver.Chrome()  # Or any other browser driver

def extract_alienvault_report(url):
    try:
        # 2. Navigate to the webpage containing the table
        driver.get(url)

        # 3. Locate the <select> element that controls the number of items
        select_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "count"))  # Replace with the actual ID
            # Or use other locators like By.NAME, By.XPATH, By.CSS_SELECTOR
        )
        item_selector = Select(select_element)

        # 4. Select the desired number of items
        desired_item_count = "100"  # Replace with the value you want to select (e.g., "50", "All")
        item_selector.select_by_value(desired_item_count)  # Or select_by_visible_text() or select_by_index()

        # 5. Wait for the table to load/update after the selection (important!)
        WebDriverWait(driver, 10)
        # .until(
        #     EC.presence_of_element_located((By.XPATH, "//table[@id='your_table_id']/tbody/tr"))  # Adjust XPath to target a row in the table
        # )
        time.sleep(2)  # Add an extra small delay if needed for rendering

        # 6. Get the updated HTML source
        html_source = driver.page_source
        soup = BeautifulSoup(html_source, 'html.parser')

        # # 7. Locate and extract the table content using BeautifulSoup
        table = soup.find("otx-table")#, {'id': 'your_table_id'})  # Replace with the actual table ID
    finally:
        # 8. Close the browser
        driver.quit()

    

In [ ]:
docs_dict = {}
for k, v in references_dict.items():
    doc = ""
    for url in v:
        try:
            report = extract_report(url)
        except Exception as e:
            report = "REPORT IS NOT EXTRACTED! Reason caused the failure:\n\n{e}".format(
                e = e
            )
        doc += "External reference URL: {url}\n\nCTI REPORT\n\n{report}\n\n".format(
            url = url,
            report = report
        )
    docs_dict.update({k:doc})
    with open(os.path.join("data/reports_docling", k), mode="w", encoding="utf-8") as f:
        f.write(doc)

Input document IOCs-Spring4Shell.txt does not match any allowed format.
c:\Users\AIAS\Documents\cti-model-training\venv-eda\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\AIAS\.cache\huggingface\hub\models--ds4sd--docling-models. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Error processing child fr

: 

: 

In [47]:
#result.document.pictures

In [48]:
source = "https://otx.alienvault.com/pulse/624c29baad734a210134b02c"
converter = DocumentConverter()
result = converter.convert(source)
result.document.export_to_markdown()

''

In [13]:
from bs4 import BeautifulSoup
from bs4.element import Tag
from selenium import webdriver
import requests
import re
import time

def get_soup_object(url, dynamic=False):
    if dynamic:
        html_content = requests.get(url)
        soup = BeautifulSoup(html_content.content, "html.parser")
    else:
        driver = webdriver.Chrome()
        driver.get(url)
        time.sleep(2)
        html_content = driver.page_source
        soup = BeautifulSoup(html_content, 'html.parser')
        driver.quit()
    return soup

In [14]:
soup = get_soup_object(source, dynamic=False)

In [49]:
title = soup.find("h1").get_text()
print(title)

State-sponsored Attack Groups Capitalise on Russia-Ukraine War for Cyber Espionage


In [51]:
metadata = soup.find("div", class_="pulse-meta").find('ul').get_text().strip()
print(metadata)

Created  3 years ago  Modified 3 years ago by AlienVault Public TLP:  White


In [50]:
contents = "\n\n".join(
    [tag.get_text().strip() for tag in soup.find("div", class_="pulse-general-details").find_all("div", class_="row col-md-12 col-flex detail-row ng-star-inserted")]
)
print(contents)

Tags: APT, spear-phishing, Ukraine, geopolitical conflict

Adversary:  El Machete, SideWinder, Lyceum

Industries:Maritime, Energy, Government, Financial

Targeted Countries: Ukraine ,  Russian Federation

Malware Families: Golang ,  SideWinder ,  Lyceum ,  El Machete

Att&ck IDs: T1566 - Phishing ,  T1127 - Trusted Developer Utilities Proxy Execution ,  T1115 - Clipboard Data ,  T1056 - Input Capture ,  T1218 - Signed Binary Proxy Execution ,  T1203 - Exploitation for Client Execution ,  T1033 - System Owner/User Discovery ,  T1104 - Multi-Stage Channels ,  T1095 - Non-Application Layer Protocol ,  T1113 - Screen Capture ,  T1564 - Hide Artifacts ,  T1132 - Data Encoding ,  T1036 - Masquerading ,  T1140 - Deobfuscate/Decode Files or Information


In [45]:
table_rows = soup.find("otx-table").find_all("tr")
for tr in table_rows:
    print(tr.get_text())

typeindicatorRoletitleAddedActiverelated Pulses
IPv431.207.44.72command_and_controlPy/CannibalApr 5, 2022, 11:36:27 AM35
domaincyberclub.onecommand_and_controlTA457Apr 5, 2022, 11:36:27 AM81
FileHash-MD514d3040db6d97f13250c438e2acb64acratPy/CannibalApr 5, 2022, 11:36:27 AM8
FileHash-SHA1c66d5875b5db70aad23c868449c3cabbf5d90d0fratPy/CannibalApr 5, 2022, 11:36:27 AM8
FileHash-SHA256ed09da9d48afe918f9c7f72fe4466167e2f127a28a7641ba80d6165e82f48431ratPy/CannibalApr 5, 2022, 11:36:27 AM8
FileHash-MD58199f14502e80581000bd5b3bda250eetrojanTA457, DeviousDroneApr 5, 2022, 11:36:27 AM18
FileHash-MD5bcb465cc2257e5777bab431690ca5039trojanTA457, DeviousDroneApr 5, 2022, 11:36:27 AM10
FileHash-MD5d79687676d2d152aec4143c852bdbc4atrojanTA457, DeviousDroneApr 5, 2022, 11:36:27 AM12
FileHash-SHA10bc47b1044d1c795ab6cb9d51003e13ade567c6dtrojanTA457, DeviousDroneApr 5, 2022, 11:36:27 AM18
FileHash-SHA15a0f97b4aa465f64e6d8f640c2ccb36b400fc68ctrojanTA457, DeviousDroneApr 5, 2022, 11:36:27 AM9


In [54]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

# 1. Initialize Selenium WebDriver (adjust path to your webdriver)
driver = webdriver.Chrome()  # Or any other browser driver

try:
    # 2. Navigate to the webpage containing the table
    url = source
    driver.get(url)

    # 3. Locate the <select> element that controls the number of items
    select_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "count"))  # Replace with the actual ID
        # Or use other locators like By.NAME, By.XPATH, By.CSS_SELECTOR
    )
    item_selector = Select(select_element)

    # 4. Select the desired number of items
    desired_item_count = "100"  # Replace with the value you want to select (e.g., "50", "All")
    item_selector.select_by_value(desired_item_count)  # Or select_by_visible_text() or select_by_index()

    # 5. Wait for the table to load/update after the selection (important!)
    WebDriverWait(driver, 10)
    # .until(
    #     EC.presence_of_element_located((By.XPATH, "//table[@id='your_table_id']/tbody/tr"))  # Adjust XPath to target a row in the table
    # )
    time.sleep(2)  # Add an extra small delay if needed for rendering

    # 6. Get the updated HTML source
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'html.parser')

    # 7. Locate and extract the table content using BeautifulSoup
    table = soup.find("otx-table")#, {'id': 'your_table_id'})  # Replace with the actual table ID

    # if table:
    #     # Extract headers
    #     headers = [th.text.strip() for th in table.find('thead').find_all('th')] if table.find('thead') else []

    #     # Extract rows
    #     rows = []
    #     tbody = table.find('tbody')
    #     if tbody:
    #         for tr in tbody.find_all('tr'):
    #             row_data = [td.text.strip() for td in tr.find_all('td')]
    #             rows.append(row_data)

    #     # Process the extracted data (headers and rows)
    #     print("Headers:", headers)
    #     print("Rows:", rows)
    # else:
    #     print("Table not found.")

finally:
    # 8. Close the browser
    driver.quit()

In [59]:
table_rows = table.find_all("tr")

In [60]:
table_rows[0]

<tr _ngcontent-apd-c167=""><!-- --><th _ngcontent-apd-c167="" class="clickable ng-star-inserted" style="width: 115px;"><div _ngcontent-apd-c167="" class="column-header-container"><div _ngcontent-apd-c167="" class="clickable">type</div><otx-table-sort-icon _ngcontent-apd-c167="" _nghost-apd-c112=""><div _ngcontent-apd-c112="" class="sort-icon ng-star-inserted"><!-- --><i _ngcontent-apd-c112="" class="fa fa-sort ng-star-inserted"></i><!-- --><!-- --></div><!-- --><!-- --></otx-table-sort-icon></div></th><!-- --><th _ngcontent-apd-c167="" class="clickable ng-star-inserted" style="width: 470px;"><div _ngcontent-apd-c167="" class="column-header-container"><div _ngcontent-apd-c167="" class="clickable">indicator</div><otx-table-sort-icon _ngcontent-apd-c167="" _nghost-apd-c112=""><div _ngcontent-apd-c112="" class="sort-icon ng-star-inserted"><!-- --><i _ngcontent-apd-c112="" class="fa fa-sort ng-star-inserted"></i><!-- --><!-- --></div><!-- --><!-- --></otx-table-sort-icon></div></th><!-- -->

In [68]:
table_dict = {}
for i, tr in enumerate(table_rows):
    if i==0:
        table_columns = []
        for tag in tr.find_all("div", class_="clickable"):
            table_dict.update({tag.get_text().strip():[]})
            table_columns.append(tag.get_text().strip())
    else:
        for j, tag in enumerate(tr.find_all("td", class_="ng-star-inserted")):
            k = table_columns[j]
            table_dict[k].append(tag.get_text().strip())

In [72]:
table_dict.pop("")

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [83]:
print(pd.DataFrame(table_dict).to_string(index=False, header=True, na_rep='--', col_space=20))

                type                                                        indicator                 Role                                title                    Added               Active       related Pulses
                IPv4                                                     31.207.44.72  command_and_control                          Py/Cannibal Apr 5, 2022, 11:36:27 AM                                        35
              domain                                                    cyberclub.one  command_and_control                                TA457 Apr 5, 2022, 11:36:27 AM                                        81
        FileHash-MD5                                 14d3040db6d97f13250c438e2acb64ac                  rat                          Py/Cannibal Apr 5, 2022, 11:36:27 AM                                         8
       FileHash-SHA1                         c66d5875b5db70aad23c868449c3cabbf5d90d0f                  rat                          Py/Cannibal Apr 5, 2022,

In [62]:
for tag in table_rows[0].find_all("div", class_="clickable"):
    print(tag.get_text().strip())

type
indicator
Role
title
Added
Active
related Pulses



In [64]:
for tag in table_rows[1].find_all("td", class_="ng-star-inserted"):
    print(tag.get_text().strip())

IPv4
31.207.44.72
command_and_control
Py/Cannibal
Apr 5, 2022, 11:36:27 AM

35



In [65]:
tag.get_text()

''